### Manipulation of PEAKS de novo results of Trocas 8 (low water, November 2019) non-incubation samples LC-MS/MS data using python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >50% ALC

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 675) for another
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [1]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [2]:
    cd /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/677_Trocas8_June2021_DENOVO_104/

/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/677_Trocas8_June2021_DENOVO_104


In [3]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks677 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/T8-June2021-UWPR-DN50_10lgP15/677_Trocas8_June2021_DENOVO_104/677_Trocas8_June2021_DENOVO_104_DN50.csv")

print("# redundant Peaks peptides >50% ALC in combined dataframe:", len(peaks677))

print(peaks677.columns)

# These columns mess things up- get rid of them

del peaks677['Fraction']
del peaks677['Scan']
del peaks677['Source File']
del peaks677['Tag Length']
del peaks677['PTM']
del peaks677['tag (>=0%)']
del peaks677['mode']
del peaks677['local confidence (%)']


columns = ['Peptide', 'ALC', 'length', 'm/z', 'z', 'RT', 'Area',
       'Mass', 'ppm']

peaks677.columns = columns

mean_len = peaks677['length'].mean()
print(mean_len)

# look at the dataframe
peaks677.head()

# redundant Peaks peptides >50% ALC in combined dataframe: 84
Index(['Fraction', 'Scan', 'Source File', 'Peptide', 'Tag Length', 'ALC (%)',
       'length', 'm/z', 'z', 'RT', 'Area', 'Mass', 'ppm', 'PTM',
       'local confidence (%)', 'tag (>=0%)', 'mode'],
      dtype='object')
9.369047619047619


,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm
0,YLYELAR,98,7,464.2498,2,68.64,23200.0,926.4861,-1.2
1,LSSPATLNSR,98,10,523.2846,2,44.52,14600000.0,1044.5564,-1.7
2,LSSPATLDSR,98,10,523.7767,2,47.54,141000.0,1045.5403,-1.4
3,LSSPATLNSR,98,10,523.2853,2,44.01,14600000.0,1044.5564,-0.3
4,LATVLSPR,97,8,428.7657,2,58.31,4200000.0,855.5178,-1.1


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine: 15.9949 M
    variable deamidation of asparagine, glumatine: 0.984016 NQ

We'll then write this manipulated dataframe to a new file.

In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks677['A'] = peaks677['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks677['C'] = peaks677['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks677['D'] = peaks677['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks677['E'] = peaks677['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks677['F'] = peaks677['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks677['G'] = peaks677['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks677['H'] = peaks677['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks677 output, there will be no isoleucines (they're lumped in with leucines)
peaks677['I'] = peaks677['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks677['K'] = peaks677['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks677['L'] = peaks677['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks677['M'] = peaks677['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks677['N'] = peaks677['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks677['P'] = peaks677['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks677['Q'] = peaks677['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks677['R'] = peaks677['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks677['S'] = peaks677['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks677['T'] = peaks677['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks677['V'] = peaks677['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks677['W'] = peaks677['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks677['Y'] = peaks677['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks677['c-carb'] = peaks677['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks677['m-oxid'] = peaks677['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks677['n-deam'] = peaks677['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
peaks677['q-deam'] = peaks677['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaks677['stripped_peptide'] = peaks677['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks677['stripped_length'] = peaks677['stripped_peptide'].apply(len)

# total the number of modifications in sequence
peaks677['ptm-total'] = peaks677['c-carb'] + peaks677['m-oxid'] + peaks677['n-deam'] + peaks677['q-deam']

# calculate NAAF numerator for each peptide k
peaks677['NAAF_num.'] = peaks677['Area'] / peaks677['stripped_length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks677.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/677_CV_50_GF75_DN/677_CV_50_GF75_DN50.csv")

# check out the results
peaks677.head()

,Peptide,ALC,length,m/z,z,RT,Area,Mass,ppm,A,...,W,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,ptm-total,NAAF_num.
0,YLYELAR,98,7,464.2498,2,68.64,23200.0,926.4861,-1.2,1,...,0,2,0,0,0,0,YLYELAR,7,0,3.314286e+03
1,LSSPATLNSR,98,10,523.2846,2,44.52,14600000.0,1044.5564,-1.7,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,1.460000e+06
2,LSSPATLDSR,98,10,523.7767,2,47.54,141000.0,1045.5403,-1.4,1,...,0,0,0,0,0,0,LSSPATLDSR,10,0,1.410000e+04
3,LSSPATLNSR,98,10,523.2853,2,44.01,14600000.0,1044.5564,-0.3,1,...,0,0,0,0,0,0,LSSPATLNSR,10,0,1.460000e+06
4,LATVLSPR,97,8,428.7657,2,58.31,4200000.0,855.5178,-1.1,1,...,0,0,0,0,0,0,LATVLSPR,8,0,5.250000e+05


In [7]:
# keep only stripped peptide I/L and NAAF
dn_677 = peaks677[['stripped_peptide', 'Area', 'NAAF_num.']]

dn_677.set_index('stripped_peptide')

# write modified dataframe to new txt file
dn_677.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/I-L_NAAFs/677_CV_50_GF75_DN50_ILnaafs.csv")

dn_677.head()

,stripped_peptide,Area,NAAF_num.
0,YLYELAR,23200.0,3.314286e+03
1,LSSPATLNSR,14600000.0,1.460000e+06
2,LSSPATLDSR,141000.0,1.410000e+04
3,LSSPATLNSR,14600000.0,1.460000e+06
4,LATVLSPR,4200000.0,5.250000e+05


In [8]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >50% ALC)

index = ['sample total']

data = {'A': peaks677['A'].sum(),
        'C': peaks677['C'].sum(),
        'D': peaks677['D'].sum(),
        'E': peaks677['E'].sum(),
        'F': peaks677['F'].sum(),
        'G': peaks677['G'].sum(),
        'H': peaks677['H'].sum(),
        'I': peaks677['I'].sum(),
        'K': peaks677['K'].sum(),
        'L': peaks677['L'].sum(),
        'M': peaks677['M'].sum(),
        'N': peaks677['N'].sum(),
        'P': peaks677['P'].sum(),
        'Q': peaks677['Q'].sum(),
        'R': peaks677['R'].sum(),
        'S': peaks677['S'].sum(),
        'T': peaks677['T'].sum(),
        'V': peaks677['V'].sum(),
        'W': peaks677['W'].sum(),
        'Y': peaks677['Y'].sum(),
        'c-carb': peaks677['c-carb'].sum(),
        'm-oxid': peaks677['m-oxid'].sum(),
        'n-deam': peaks677['n-deam'].sum(),
        'q-deam': peaks677['q-deam'].sum(),
        'Total area': peaks677['Area'].sum(),
        'Total length': peaks677['stripped_length'].sum()
       }

totalpeaks677 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'n-deam', 'q-deam', \
                                            'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks677['% C w/ carb'] = totalpeaks677['c-carb'] / totalpeaks677['C'] 

# calculate percentage of M's that are oxidized
totalpeaks677['% M w/ oxid'] = totalpeaks677['m-oxid'] / totalpeaks677['M'] 

# calculate percentage of N's that are deamidated
totalpeaks677['% N w/ deam'] = totalpeaks677['n-deam'] / totalpeaks677['N'] 

# calculate percentage of N's that are deamidated
totalpeaks677['% Q w/ deam'] = totalpeaks677['q-deam'] / totalpeaks677['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks677['NAAF denom.'] = totalpeaks677['Total area'] / totalpeaks677['Total length']

# write modified dataframe to new txt file
totalpeaks677.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/677_CV_50_GF75_DN/677_CV_50_GF75_DN50_totals.csv")

totalpeaks677.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,64,10,15,27,15,24,44,0,36,121,...,5,5,1,1.770529e+08,771,1.0,0.116279,0.142857,0.111111,229640.580467


In [9]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF50 = 229640.580467

# use NAAF >50% ALC to get NAAF factor
peaks677['NAAF factor'] = (peaks677['NAAF_num.'])/NAAF50

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_677 = peaks677[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'n-deam', 'q-deam']].copy()

# multiply the NAAF50 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_677['A-NAAF50'] = peaksAAPTM_677['A'] * peaks677['NAAF factor']
peaksAAPTM_677['C-NAAF50'] = peaksAAPTM_677['C'] * peaks677['NAAF factor']
peaksAAPTM_677['D-NAAF50'] = peaksAAPTM_677['D'] * peaks677['NAAF factor']
peaksAAPTM_677['E-NAAF50'] = peaksAAPTM_677['E'] * peaks677['NAAF factor']
peaksAAPTM_677['F-NAAF50'] = peaksAAPTM_677['F'] * peaks677['NAAF factor']
peaksAAPTM_677['G-NAAF50'] = peaksAAPTM_677['G'] * peaks677['NAAF factor']
peaksAAPTM_677['H-NAAF50'] = peaksAAPTM_677['H'] * peaks677['NAAF factor']
peaksAAPTM_677['I-NAAF50'] = peaksAAPTM_677['I'] * peaks677['NAAF factor']
peaksAAPTM_677['K-NAAF50'] = peaksAAPTM_677['K'] * peaks677['NAAF factor']
peaksAAPTM_677['L-NAAF50'] = peaksAAPTM_677['L'] * peaks677['NAAF factor']
peaksAAPTM_677['M-NAAF50'] = peaksAAPTM_677['M'] * peaks677['NAAF factor']
peaksAAPTM_677['N-NAAF50'] = peaksAAPTM_677['N'] * peaks677['NAAF factor']
peaksAAPTM_677['P-NAAF50'] = peaksAAPTM_677['P'] * peaks677['NAAF factor']
peaksAAPTM_677['Q-NAAF50'] = peaksAAPTM_677['Q'] * peaks677['NAAF factor']
peaksAAPTM_677['R-NAAF50'] = peaksAAPTM_677['R'] * peaks677['NAAF factor']
peaksAAPTM_677['S-NAAF50'] = peaksAAPTM_677['S'] * peaks677['NAAF factor']
peaksAAPTM_677['T-NAAF50'] = peaksAAPTM_677['T'] * peaks677['NAAF factor']
peaksAAPTM_677['V-NAAF50'] = peaksAAPTM_677['V'] * peaks677['NAAF factor']
peaksAAPTM_677['W-NAAF50'] = peaksAAPTM_677['W'] * peaks677['NAAF factor']
peaksAAPTM_677['Y-NAAF50'] = peaksAAPTM_677['Y'] * peaks677['NAAF factor']

# multiply the NAAF50 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_677['ccarb-NAAF50'] = peaksAAPTM_677['c-carb'] * peaksAAPTM_677['NAAF factor']
peaksAAPTM_677['moxid-NAAF50'] = peaksAAPTM_677['m-oxid'] * peaksAAPTM_677['NAAF factor']
peaksAAPTM_677['ndeam-NAAF50'] = peaksAAPTM_677['n-deam'] * peaksAAPTM_677['NAAF factor']
peaksAAPTM_677['qdeam-NAAF50'] = peaksAAPTM_677['q-deam'] * peaksAAPTM_677['NAAF factor']


# write the dataframe to a new csv
peaksAAPTM_677.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/677_CV_50_GF75_DN/677_CV_50_GF75_DN50_naaf.csv")

peaksAAPTM_677.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,K,...,R-NAAF50,S-NAAF50,T-NAAF50,V-NAAF50,W-NAAF50,Y-NAAF50,ccarb-NAAF50,moxid-NAAF50,ndeam-NAAF50,qdeam-NAAF50
0,YLYELAR,0.014432,1,0,0,1,0,0,0,0,...,0.014432,0.000000,0.000000,0.000000,0.0,0.028865,0.0,0.0,0.0,0.0
1,LSSPATLNSR,6.357761,1,0,0,0,0,0,0,0,...,6.357761,19.073284,6.357761,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,LSSPATLDSR,0.061400,1,0,1,0,0,0,0,0,...,0.061400,0.184201,0.061400,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
3,LSSPATLNSR,6.357761,1,0,0,0,0,0,0,0,...,6.357761,19.073284,6.357761,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,LATVLSPR,2.286181,1,0,0,0,0,0,0,0,...,2.286181,2.286181,2.286181,2.286181,0.0,0.000000,0.0,0.0,0.0,0.0


In [10]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_677['NAAF factor'].sum(),
        'A': peaksAAPTM_677['A-NAAF50'].sum(),
        'C': peaksAAPTM_677['C-NAAF50'].sum(),
        'D': peaksAAPTM_677['D-NAAF50'].sum(),
        'E': peaksAAPTM_677['E-NAAF50'].sum(),
        'F': peaksAAPTM_677['F-NAAF50'].sum(),
        'G': peaksAAPTM_677['G-NAAF50'].sum(),
        'H': peaksAAPTM_677['H-NAAF50'].sum(),
        'I': peaksAAPTM_677['I-NAAF50'].sum(),
        'K': peaksAAPTM_677['K-NAAF50'].sum(),
        'L': peaksAAPTM_677['L-NAAF50'].sum(),
        'M': peaksAAPTM_677['M-NAAF50'].sum(),
        'N': peaksAAPTM_677['N-NAAF50'].sum(),
        'P': peaksAAPTM_677['P-NAAF50'].sum(),
        'Q': peaksAAPTM_677['Q-NAAF50'].sum(),
        'R': peaksAAPTM_677['R-NAAF50'].sum(),
        'S': peaksAAPTM_677['S-NAAF50'].sum(),
        'T': peaksAAPTM_677['T-NAAF50'].sum(),
        'V': peaksAAPTM_677['V-NAAF50'].sum(),
        'W': peaksAAPTM_677['W-NAAF50'].sum(),
        'Y': peaksAAPTM_677['Y-NAAF50'].sum(),
        'c-carb': peaksAAPTM_677['ccarb-NAAF50'].sum(),
        'm-oxid': peaksAAPTM_677['moxid-NAAF50'].sum(),
        'n-deam': peaksAAPTM_677['ndeam-NAAF50'].sum(),
        'q-deam': peaksAAPTM_677['qdeam-NAAF50'].sum(),
       }

totalpeaks50_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks50_NAAF['% C w/ carb.'] = totalpeaks50_NAAF['c-carb'] / totalpeaks50_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks50_NAAF['% M w/ oxid'] = totalpeaks50_NAAF['m-oxid'] / totalpeaks50_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% N w/ deam'] = totalpeaks50_NAAF['n-deam'] / totalpeaks50_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks50_NAAF['% Q w/ deam'] = totalpeaks50_NAAF['q-deam'] / totalpeaks50_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks50_NAAF['NAAF check'] = totalpeaks50_NAAF['NAAF'] / 229640.580467

# write modified dataframe to new txt file, same name + totals
totalpeaks50_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/677_CV_50_GF75_DN/677_CV_50_GF75_DN50_naaf_totals.csv")

totalpeaks50_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,89.557235,63.66508,0.332788,1.066245,0.535169,0.833013,4.259773,2.845934,0.0,1.281707,...,1.860431,0.332788,0.168963,0.322535,0.020902,1.0,0.051709,0.012229,0.174714,0.00039


## Export stripped peptides >50% ALC

In [11]:
##### keep only stripped peptide column 
pep50 = peaks677[["stripped_peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep50.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/677_CV_50_GF75_DN/677_CV_50_GF75_DN50_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/677_CV_50_GF75_DN/677_CV_50_GF75_DN50_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas8-notincs/processed/PeaksDN/677_CV_50_GF75_DN/677_CV_50_GF75_DN50_stripped_peptides.fas

# removing redundancy
peaks50dedup = pd.DataFrame.drop_duplicates(pep50)

print("# redundant stripped Peaks peptides >50% ALC", len(pep50))
print("# nonredundant stripped Peaks peptides >50% ALC", len(peaks50dedup))
print("average peptide length Peaks peptides >50% ALC", peaks677['stripped_length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >50% ALC
pep50m = peaks677[["Peptide"]]

# deduplicate
pep50mdedup = pd.DataFrame.drop_duplicates(pep50m)

print("# redundant Peaks peptides >50% ALC", len(pep50m))
print("# nonredundant Peaks peptides", len(pep50mdedup))

# check
pep50.head()

# redundant stripped Peaks peptides >50% ALC 84
# nonredundant stripped Peaks peptides >50% ALC 61
average peptide length Peaks peptides >50% ALC 9.178571428571429
# redundant Peaks peptides >50% ALC 84
# nonredundant Peaks peptides 61


,stripped_peptide
0,YLYELAR
1,LSSPATLNSR
2,LSSPATLDSR
3,LSSPATLNSR
4,LATVLSPR
